In [1]:
'''Trains a simple binarize fully connected NN on the MNIST dataset.
Modified from keras' examples/mnist_mlp.py
Gets to 97.9% test accuracy after 20 epochs using theano backend
'''


from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for repeatability

In [2]:
import tensorflow as tf

import numpy as np
import os
import sys
import classifier
from classifier import KerasClassifier
import pickle as picklerick
import gzip
import urllib.request
import tensorflow.keras.backend as K
from tensorflow.keras.losses import squared_hinge
from tensorflow.keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
 
from keras.losses import losses_utils
from keras.losses import SquaredHinge
from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, Clip



from keras.models import load_model



2023-02-06 10:44:00.120389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 10:44:00.193589: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 10:44:00.195419: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 10:44:00.195426: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [3]:
from art.attacks.evasion import CarliniLInfMethod

from art.utils import load_mnist

In [4]:
class DropoutNoScale(Dropout):
    '''Keras Dropout does scale the input in training phase, which is undesirable here.
    '''
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed) * (1 - self.rate)
            return K.in_train_phase(dropped_inputs, inputs,
                                    training=training)
        return inputs

In [5]:
def binary_tanh(x):
    return binary_tanh_op(x)

In [6]:
batch_size = 100
epochs = 2 # adjust for accuracy (max accuracy = ~97%)
nb_classes = 10

H = 'Glorot'
kernel_lr_multiplier = 'Glorot'

In [7]:
# network

num_unit = 2048
num_hidden = 3
use_bias = False

In [8]:
#learning rate schedule

lr_start = 1e-3
lr_end = 1e-6
lr_decay = (lr_end / lr_start)**(1. / epochs)

In [9]:
#BN
epsilon = 1e-6
momentum = 0.9

In [10]:
#dropout
drop_in =0.2
drop_hidden = 0.5

In [11]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [12]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes) * 2 - 1 # -1 or 1 for hinge loss
Y_test = np_utils.to_categorical(y_test, nb_classes) * 2 - 1

print(Y_train.shape[0], 'train samples')
print(Y_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [13]:
model = Sequential()
model.add(DropoutNoScale(drop_in, input_shape=(784,), name='drop0'))
for i in range(num_hidden):
    
    model.add(BinaryDense(num_unit, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias,
              name='dense{}'.format(i+1)))
    
    model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn{}'.format(i+1)))
    
    model.add(Activation(binary_tanh, name='act{}'.format(i+1)))
    
    model.add(DropoutNoScale(drop_hidden, name='drop{}'.format(i+1)))

model.add(BinaryDense(10, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias,
          name='dense'))

model.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn'))


model.summary()


opt = Adam(lr=lr_start) 

model.compile(loss = tf.keras.losses.squared_hinge, optimizer=opt, metrics=['acc'])

#model.compile(loss = 'squared_hinge', optimizer=opt, metrics=['acc'])


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-06 10:44:01.172695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-06 10:44:01.172793: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 10:44:01.172813: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-06 10:44:01.172828: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-06 10:44:01.184667: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 drop0 (DropoutNoScale)      (None, 784)               0         
                                                                 
 dense1 (BinaryDense)        (None, 2048)              1605632   
                                                                 
 bn1 (BatchNormalization)    (None, 2048)              8192      
                                                                 
 act1 (Activation)           (None, 2048)              0         
                                                                 
 drop1 (DropoutNoScale)      (None, 2048)              0         
                                                                 
 dense2 (BinaryDense)        (None, 2048)              4194304   
                                                                 
 bn2 (BatchNormalization)    (None, 2048)              8

/home/aaronlaszlo/miniconda3/envs/robust/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
lr_scheduler = LearningRateScheduler(lambda e: lr_start * lr_decay ** e)
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[lr_scheduler])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
600/600 [==============================] - 27s 45ms/step - loss: 0.3965 - acc: 0.8779 - val_loss: 0.1344 - val_acc: 0.9412 - lr: 0.0010
Epoch 2/2
600/600 [==============================] - 27s 44ms/step - loss: 0.1407 - acc: 0.9189 - val_loss: 0.1129 - val_acc: 0.9459 - lr: 3.1623e-05
Test score: 0.11293046921491623
Test accuracy: 0.945900022983551


In [15]:
picklerick.dump(model, open('model.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_2
.........vars
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......batch_normalization_3
.........vars
............0
............1
............2
............3
......binary_dense
.........vars
............0
......binary_dense_1
.........vars
............0
......binary_dense_2
.........vars
............0
......binary_dense_3
.........vars
............0
......dropout_no_scale
.........vars
......dropout_no_scale_1
.........vars
......dropout_no_scale_2
.........vars
......dropout_no_scale_3
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper

In [16]:

for i in range(0, 20):
    weights = model.get_weights()[i] # returs a numpy list of weights
    print("Layer: " )
    laytype = model.layers[i].get_config()
    print(laytype)
    print(i+1)
    print(weights)
    print('\n')
    


Layer: 
{'name': 'drop0', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 784), 'rate': 0.2, 'noise_shape': None, 'seed': None}
1
[[-6.8975538e-03 -6.3410792e-03  2.1085287e-02 ...  2.0794557e-02
   1.2606410e-02  4.4544879e-03]
 [ 1.1027535e-02  1.3243550e-02 -1.4922472e-02 ... -1.4710017e-03
  -1.9780850e-02  2.6061740e-03]
 [-5.4968521e-03 -1.3674755e-02 -4.3027103e-06 ... -5.2581485e-03
   1.4351627e-02  1.9307142e-02]
 ...
 [-1.6804162e-02 -1.1528949e-02  6.6673197e-04 ... -1.1268199e-02
   1.8686896e-02 -1.0574647e-02]
 [ 1.7852409e-02 -1.5845915e-02  2.0837104e-02 ... -1.3290688e-02
  -4.0026996e-03  5.0387494e-03]
 [ 2.0643109e-02  1.9593274e-02 -6.6646934e-03 ...  8.1234481e-03
   1.5896363e-02 -7.8744246e-03]]


Layer: 
{'name': 'dense1', 'trainable': True, 'dtype': 'float32', 'units': 2048, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'RandomUniform', 'config': {'minval': -0.023014365, 'maxval': 0.023014365, 'seed': None}

IndexError: list index out of range

In [ ]:
def show(img)